In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import utils as ut
random_seed = 10

In [2]:
train_features = pd.read_csv("DOTA2_TRAIN_features.csv")
train_targets = pd.read_csv("DOTA2_TRAIN_targets.csv")
train = train_features.copy()
train["target"] = train_targets["radiant_win"]
X_train,Y_train, X_test, Y_test = ut.preprocess_train(train, True)

In [3]:
X_train

,game_time,game_mode,lobby_type,objectives_len,chat_len,r1_hero_id,r1_kills,r1_deaths,r1_assists,r1_denies,...,d5_stuns,d5_creeps_stacked,d5_camps_stacked,d5_rune_pickups,d5_firstblood_claimed,d5_teamfight_participation,d5_towers_killed,d5_roshans_killed,d5_obs_placed,d5_sen_placed
7676,675.0,22.0,7.0,1.0,0.0,74.0,0.0,1.0,0.0,10.0,...,12.997,0.0,0.0,2.0,0.0,0.667,0.0,0.0,0.0,0.0
1257,203.0,22.0,7.0,1.0,7.0,19.0,0.0,0.0,0.0,4.0,...,2.433,0.0,0.0,1.0,0.0,0.167,0.0,0.0,1.0,0.0
24896,1129.0,22.0,7.0,4.0,10.0,5.0,2.0,7.0,6.0,5.0,...,11.331,0.0,0.0,4.0,0.0,0.312,1.0,0.0,0.0,0.0
7056,1768.0,22.0,7.0,8.0,7.0,120.0,1.0,4.0,6.0,11.0,...,0.000,3.0,2.0,3.0,0.0,0.400,0.0,1.0,0.0,0.0
21579,1489.0,22.0,7.0,8.0,10.0,88.0,2.0,5.0,2.0,4.0,...,10.631,15.0,4.0,8.0,0.0,0.409,0.0,0.0,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23444,2434.0,22.0,0.0,16.0,0.0,91.0,2.0,10.0,20.0,0.0,...,1.733,0.0,0.0,2.0,0.0,0.591,0.0,0.0,9.0,2.0
9392,1040.0,22.0,7.0,4.0,12.0,12.0,8.0,4.0,7.0,6.0,...,7.598,0.0,0.0,0.0,0.0,0.158,0.0,0.0,0.0,0.0
1499,2102.0,22.0,7.0,10.0,23.0,41.0,4.0,12.0,3.0,2.0,...,0.000,0.0,0.0,6.0,1.0,0.500,0.0,0.0,11.0,14.0
27569,1148.0,22.0,7.0,4.0,4.0,62.0,1.0,7.0,7.0,0.0,...,0.000,3.0,1.0,4.0,0.0,0.188,0.0,0.0,2.0,3.0


In [4]:
from sklearn import tree
from catboost import CatBoostClassifier, metrics
#parameters = {
#    'criterion' : ['entropy'],
#}
model = CatBoostClassifier(random_seed=random_seed, loss_function = "Logloss")
#model = GridSearchCV(cf, param_grid=parameters, cv= 5)
model.fit(X_train, Y_train)

Learning rate set to 0.03051
0:	learn: 0.6885190	total: 234ms	remaining: 3m 54s
1:	learn: 0.6841930	total: 288ms	remaining: 2m 23s
2:	learn: 0.6796884	total: 339ms	remaining: 1m 52s
3:	learn: 0.6755415	total: 383ms	remaining: 1m 35s
4:	learn: 0.6714683	total: 425ms	remaining: 1m 24s
5:	learn: 0.6675781	total: 461ms	remaining: 1m 16s
6:	learn: 0.6642646	total: 500ms	remaining: 1m 10s
7:	learn: 0.6608526	total: 537ms	remaining: 1m 6s
8:	learn: 0.6576357	total: 575ms	remaining: 1m 3s
9:	learn: 0.6546956	total: 611ms	remaining: 1m
10:	learn: 0.6518965	total: 653ms	remaining: 58.7s
11:	learn: 0.6493373	total: 688ms	remaining: 56.6s
12:	learn: 0.6463674	total: 724ms	remaining: 55s
13:	learn: 0.6435773	total: 760ms	remaining: 53.5s
14:	learn: 0.6407610	total: 797ms	remaining: 52.3s
15:	learn: 0.6383779	total: 834ms	remaining: 51.3s
16:	learn: 0.6360914	total: 883ms	remaining: 51.1s
17:	learn: 0.6337924	total: 915ms	remaining: 49.9s
18:	learn: 0.6314661	total: 948ms	remaining: 49s
19:	learn: 0

In [5]:
#model.best_params_

In [6]:
from sklearn.metrics import roc_auc_score
print(f"Score : {model.score(X_test, Y_test)}")
print(f"ROC-AUC score : {roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])}")

Score : 0.7165775401069518
ROC-AUC score : 0.8049990521131531


In [7]:
model.predict_proba(X_test)[:1]

array([[0.65193022, 0.34806978]])

In [8]:
ut.write(model)